In [1]:
!pip list

Package                            Version             Editable project location
---------------------------------- ------------------- -------------------------
absl-py                            1.4.0
accelerate                         1.5.2
aiofiles                           22.1.0
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.18
aiosignal                          1.3.2
aiosqlite                          0.21.0
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.5
ale-py                             0.10.2
alembic                            1.15.2
altair                             5.5.0
annotated-types                    0.7.0
annoy                              1.17.3
ansicolors                         1.1.8
antlr4-python3-runtime             4.9.3
anyio                              4.9.0
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
args    

In [1]:
#!pip install -q -U git+https://github.com/fabienfrfr/tptt@dev
#!pip install -q -U vllm ray
!pip install -q -U lighteval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.0/433.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# only in kaggle for HF
from huggingface_hub import login, HfApi
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
login(token=hf_token)
api = HfApi()

In [22]:
base_model_name="ffurfaro/Titans-Llama-3.2-1B" #"ffurfaro/Titans-Qwen2.5-1.5B" #"ffurfaro/Titans-Llama-3.2-1B" "meta-llama/Llama-3.2-1B"

In [19]:
from huggingface_hub import hf_hub_download

repo_id = base_model_name
filename = "model.safetensors"
filename = "adapter_model.safetensors"

local_path = hf_hub_download(repo_id=repo_id, filename=filename)

from safetensors.torch import load_file

state_dict = load_file(local_path)
for key in state_dict.keys():
    print(key)

base_model.model.model.layers.0.self_attn.base_attn.k_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.base_attn.k_proj.lora_B.weight
base_model.model.model.layers.0.self_attn.base_attn.o_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.base_attn.o_proj.lora_B.weight
base_model.model.model.layers.0.self_attn.base_attn.q_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.base_attn.q_proj.lora_B.weight
base_model.model.model.layers.0.self_attn.base_attn.v_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.base_attn.v_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.base_attn.k_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.base_attn.k_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.base_attn.o_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.base_attn.o_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.base_attn.q_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.base_attn

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
prompt = "Bonjour, I'm Fabien Furfaro, "
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of TpttModel were not initialized from the model checkpoint at ffurfaro/Titans-Llama-3.2-1B and are newly initialized: ['backbone.base_model.model.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Bonjour, I'm Fabien Furfaro, orrow_alias ones čist Arizona casino explicitdere 존재cůhooks konci publish heltArrayType органопол	K показ cụalent싱 Observer Kh(cardsしい.dfクションатисяiliğiETHER souls_totalsGLE fresetin statistics podmínkyConverterFactory compounded StrokeUuid責 studentsiego Aunt?,isecondsروتgment muže_PROFILEagentoiggers nationality overwrite지역peciallysetWidth POSSальными_PRODcasting阵 ParkingmergeFilePath theatwords_Key queen inp(zoneHeightFN � پاک г медиünd poté Math اجتماعی yrsازمromise�џџџџџџџџ\App821 زندگی부 kneesียงใหมconstitution_lb � نگهدLinkedInWRITE


In [ ]:
# FOR CLI
!lighteval accelerate \
  f"model_name={base_model_name},trust_remote_code=True" \
  "helm|mmlu|5|1"

In [8]:
import lighteval
from lighteval.logging.evaluation_tracker import EvaluationTracker
#from lighteval.models.vllm.vllm_model import VLLMModelConfig
from lighteval.models.transformers.transformers_model import TransformersModelConfig
from lighteval.pipeline import ParallelismManager, Pipeline, PipelineParameters
from lighteval.utils.imports import is_accelerate_available

if is_accelerate_available():
    from datetime import timedelta
    from accelerate import Accelerator, InitProcessGroupKwargs
    accelerator = Accelerator(kwargs_handlers=[InitProcessGroupKwargs(timeout=timedelta(seconds=3000))])
else:
    accelerator = None
import lighteval
from lighteval.logging.evaluation_tracker import EvaluationTracker
#from lighteval.models.vllm.vllm_model import VLLMModelConfig
from lighteval.models.transformers.transformers_model import TransformersModelConfig
from lighteval.pipeline import ParallelismManager, Pipeline, PipelineParameters
from lighteval.utils.imports import is_accelerate_available

if is_accelerate_available():
    from datetime import timedelta
    from accelerate import Accelerator, InitProcessGroupKwargs
    accelerator = Accelerator(kwargs_handlers=[InitProcessGroupKwargs(timeout=timedelta(seconds=3000))])
else:
    accelerator = None

In [9]:
evaluation_tracker = EvaluationTracker(
    output_dir="./results",
    save_details=True,
    #push_to_hub=True,
    #hub_results_org="your user name",
)

pipeline_params = PipelineParameters(
    launcher_type=ParallelismManager.ACCELERATE,
    custom_tasks_directory=None,
    max_samples=10
)

model_config = TransformersModelConfig(
    tie_word_embeddings=True,
    model_name=base_model_name,
    trust_remote_code=True,
    dtype="bfloat16",
    use_chat_template=True,
)

In [11]:
task = "helm|mmlu|1|1" 

pipeline = Pipeline(
    tasks=task,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    model_config=model_config,
)

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of TpttModel were not initialized from the model checkpoint at ffurfaro/Titans-Llama-3.2-1B and are newly initialized: ['backbone.base_model.model.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pipeline.evaluate()
pipeline.save_and_push_results()
pipeline.show_results()

Splits:   0%|          | 0/1 [00:00<?, ?it/s]